In [1]:
import requests
from requests import ConnectionError,HTTPError,Timeout

url = 'https://data.ntpc.gov.tw/api/datasets/71CD1490-A2DF-4198-BEF1-318479775E8A/json?page=0&size=1000'
try:
    response = requests.get(url)
    response.raise_for_status()
except ConnectionError:
    print('找不到伺服器')
except HTTPError:
    print('網頁找不到')
except Timeout:
    print('超過時間沒有回應')
else:
    print('沒有發生錯誤')

沒有發生錯誤


In [9]:
import numpy as np
import pandas as pd
records=response.json()
dataFrame=pd.DataFrame(records,columns=['sna', 'tot', 'sbi', 'sarea', 'ar', 'bemp'])
dataFrame=dataFrame.rename(columns={'sna':'站名', 'tot':'數量', 'sbi':'可借', 'sarea':'區域','ar':'地址', 'bemp':'可還'})
dataFrame

,站名,數量,可借,區域,地址,可還
0,大鵬華城,38,14,新店區,新北市新店區中正路700巷3號,24
1,汐止火車站,56,48,汐止區,南昌街/新昌路口(西側廣場),6
2,汐止區公所,46,12,汐止區,新台五路一段/仁愛路口(新台五路側汐止地政事務所前機車停車場),34
3,國泰綜合醫院,56,21,汐止區,建成路78號對面停車場,34
4,裕隆公園,40,22,新店區,寶中路/品牌路口(東南側),18
...,...,...,...,...,...,...
662,中山溫泉公園,50,0,金山區,民生路204號(旁),0
663,學士學府路口(大苑社區),40,0,土城區,學士路/學府路(西北側建案基地),0
664,龍米停車場,36,1,八里區,龍米路二段150號(旁),34
665,雙和醫院,30,14,中和區,中正路281號(公車亭旁),15


In [10]:
dataFrame.dtypes

站名    object
數量    object
可借    object
區域    object
地址    object
可還    object
dtype: object

In [14]:
dataFrame[['數量','可借','可還']] = dataFrame[['數量','可借','可還']].astype(int)
dataFrame

,站名,數量,可借,區域,地址,可還
0,大鵬華城,38,14,新店區,新北市新店區中正路700巷3號,24
1,汐止火車站,56,48,汐止區,南昌街/新昌路口(西側廣場),6
2,汐止區公所,46,12,汐止區,新台五路一段/仁愛路口(新台五路側汐止地政事務所前機車停車場),34
3,國泰綜合醫院,56,21,汐止區,建成路78號對面停車場,34
4,裕隆公園,40,22,新店區,寶中路/品牌路口(東南側),18
...,...,...,...,...,...,...
662,中山溫泉公園,50,0,金山區,民生路204號(旁),0
663,學士學府路口(大苑社區),40,0,土城區,學士路/學府路(西北側建案基地),0
664,龍米停車場,36,1,八里區,龍米路二段150號(旁),34
665,雙和醫院,30,14,中和區,中正路281號(公車亭旁),15


In [16]:
dataFrame['故障'] = dataFrame['數量'] - (dataFrame['可借']+dataFrame['可還'])
dataFrame['故障率'] = dataFrame['故障'] / dataFrame['數量']
dataFrame

,站名,數量,可借,區域,地址,可還,故障,故障率
0,大鵬華城,38,14,新店區,新北市新店區中正路700巷3號,24,0,0.000000
1,汐止火車站,56,48,汐止區,南昌街/新昌路口(西側廣場),6,2,0.035714
2,汐止區公所,46,12,汐止區,新台五路一段/仁愛路口(新台五路側汐止地政事務所前機車停車場),34,0,0.000000
3,國泰綜合醫院,56,21,汐止區,建成路78號對面停車場,34,1,0.017857
4,裕隆公園,40,22,新店區,寶中路/品牌路口(東南側),18,0,0.000000
...,...,...,...,...,...,...,...,...
662,中山溫泉公園,50,0,金山區,民生路204號(旁),0,50,1.000000
663,學士學府路口(大苑社區),40,0,土城區,學士路/學府路(西北側建案基地),0,40,1.000000
664,龍米停車場,36,1,八里區,龍米路二段150號(旁),34,1,0.027778
665,雙和醫院,30,14,中和區,中正路281號(公車亭旁),15,1,0.033333


In [18]:
grouped=dataFrame.groupby('區域')
grouped[['數量','可還','可借','故障']].sum()

,數量,可還,可借,故障
區域,,,,
三峽區,798,450,336,12
三芝區,50,40,10,0
三重區,2678,1603,1005,70
中和區,2218,1190,902,126
五股區,706,360,339,7
八里區,260,195,59,6
土城區,1644,887,614,143
新店區,2374,1438,843,93
新莊區,2638,1392,1158,88


In [21]:
grouped[['數量','可還','可借','故障']].agg([('總量','sum'),('平均','mean'),('最大','max'),('最小','min')])

數量                        可還                       可借                 \
       總量         平均   最大  最小    總量         平均  最大  最小    總量         平均  最大   
區域                                                                            
三峽區   798  42.000000   66  30   450  23.684211  48  11   336  17.684211  50   
三芝區    50  50.000000   50  50    40  40.000000  40  40    10  10.000000  10   
三重區  2678  39.970149   88  26  1603  23.925373  52   0  1005  15.000000  43   
中和區  2218  41.849057   82  28  1190  22.452830  49   0   902  17.018868  41   
五股區   706  35.300000   48  26   360  18.000000  30   7   339  16.950000  33   
八里區   260  37.142857   46  30   195  27.857143  46  13    59   8.428571  27   
土城區  1644  42.153846   80  28   887  22.743590  47   0   614  15.743590  45   
新店區  2374  43.163636  100  26  1438  26.145455  86   0   843  15.327273  51   
新莊區  2638  41.218750  114  16  1392  21.750000  51   0  1158  18.093750  80   
板橋區  4622  44.019048  102  30  2535  24.142857  50  -2  1848  17.600000  77   
林口區  1238  37.515152   52  16   655  19.848485  28   9   571  17.303030  40   
樹林區  1352  37.555556   66  30   815  22.638889  45   9   519  14.416667  33   
永和區  1010  38.846154   66  28   643  24.730769  54   8   345  13.269231  44   
汐止區  1860  48.947368  100  28  1221  32.131579  68   0   563  14.815789  48   
泰山區   462  38.500000   64  30   275  22.916667  37   7   184  15.333333  27   
淡水區  1296  40.500000  106  26   776  24.250000  86   2   507  15.843750  28   
深坑區   134  33.500000   50  16    72  18.000000  29   6    62  15.500000  21   
瑞芳區   160  40.000000   48  30   132  33.000000  48  23    27   6.750000  15   
石門區    50  50.000000   50  50    38  38.000000  38  38    12  12.000000  12   
萬里區    50  50.000000   50  50    41  41.000000  41  41     9   9.000000   9   
蘆洲區  1164  43.111111   92  30   772  28.592593  62  14   379  14.037037  43   
金山區   186  37.200000   50  28   103  20.600000  31   0    32   6.400000  11   
鶯歌區   702  39.000000   58  30   400  22.222222  37   0   205  11.388889  25   

          故障                    
     最小   總量         平均  最大 最小  
區域                              
三峽區   8   12   0.631579   5  0  
三芝區  10    0   0.000000   0  0  
三重區   0   70   1.044776  34  0  
中和區   0  126   2.377358  64  0  
五股區   5    7   0.350000   3  0  
八里區   0    6   0.857143   2  0  
土城區   0  143   3.666667  40  0  
新店區   0   93   1.690909  40  0  
新莊區   0   88   1.375000  30  0  
板橋區   0  239   2.276190  52  0  
林口區   6   12   0.363636   3  0  
樹林區   0   18   0.500000   3  0  
永和區   2   22   0.846154   6  0  
汐止區   0   76   2.000000  40  0  
泰山區   2    3   0.250000   1  0  
淡水區   0   13   0.406250   2  0  
深坑區  10    0   0.000000   0  0  
瑞芳區   0    1   0.250000   1  0  
石門區  12    0   0.000000   0  0  
萬里區   9    0   0.000000   0  0  
蘆洲區   0   13   0.481481   4  0  
金山區   0   51  10.200000  50  0  
鶯歌區   0   97   5.388889  50  0